# Week 9 과제 1 - Tobigs 15기 이성범

**잎 사진을 통한 질병 분류**

*   데이터: 21개의 클래스로 구분된 약 20000장의 이미지
><img src="https://drive.google.com/uc?id=1YQkxnNy61Gyi3Gp6ylCKeS72BVruJXr_" width="700" height="500"> 
*   데이터 전처리
    *   전체 데이터를 train,validation,test로 분할해주세요
    *   저는 6:2:2로 분할했는데, 비율은 바꾸셔도 무방합니다.
   
    
*   학습 진행 방향
    *   Baseline 모델(pre-trained model 사용X) 구축
    *   Pre-trained 모델 사용
    *   Baseline과 ResNet50 모델의 성능을 비교해주세요 ~!

In [1]:
# gdrive에 mount
from google.colab import drive
drive.mount('/content/gdrive')
# 경로 설정
import os
os.chdir('/content/gdrive/My Drive/')

Mounted at /content/gdrive


In [ ]:
# 파이썬으로 알집 푸는 법
# !unzip -qq "./Tobigs/tobigs_week9_plant_leaf_data.zip"

## 0. 데이터 분할

* 데이터 분할을 위한 디렉토리 생성

In [2]:
import os
import shutil

original_dataset_dir = './Tobigs/tobigs_week9_plant_leaf'
classes_list = os.listdir(original_dataset_dir)

base_dir = './splitted'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for cls in classes_list:
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(validation_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))

* 데이터 분할과 클래스별 데이터 수 확인

In [3]:
import math

for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)
 
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    train_fnames = fnames[:train_size]
    print("Train size(",cls,"): ", len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print("Validation size(",cls,"): ", len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]

    print("Test size(",cls,"): ", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)

Train size( Corn___Common_rust ):  715
Validation size( Corn___Common_rust ):  238
Test size( Corn___Common_rust ):  238
Train size( Cherry___healthy ):  512
Validation size( Cherry___healthy ):  170
Test size( Cherry___healthy ):  170
Train size( Apple___Black_rot ):  372
Validation size( Apple___Black_rot ):  124
Test size( Apple___Black_rot ):  124
Train size( Apple___Apple_scab ):  378
Validation size( Apple___Apple_scab ):  126
Test size( Apple___Apple_scab ):  126
Train size( Apple___healthy ):  987
Validation size( Apple___healthy ):  329
Test size( Apple___healthy ):  329
Train size( Corn___healthy ):  697
Validation size( Corn___healthy ):  232
Test size( Corn___healthy ):  232
Train size( Apple___Cedar_apple_rust ):  165
Validation size( Apple___Cedar_apple_rust ):  55
Test size( Apple___Cedar_apple_rust ):  55
Train size( Cherry___Powdery_mildew ):  631
Validation size( Cherry___Powdery_mildew ):  210
Test size( Cherry___Powdery_mildew ):  210
Train size( Corn___Cercospora_l

## 1. 베이스라인 모델을 구축해 주세요
* Pre-Trained Model을 사용하지 않고 직접 모델을 구축해 주세요 !


In [4]:
import torch
import os

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 256
EPOCH = 30

In [5]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base)
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)

In [6]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


* 베이스라인 모델 설계하기

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class BasicBlock(nn.Module):
  def __init__(self, in_planes, planes, stride = 1):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_planes, planes,
                           kernel_size = 3,
                           stride = stride,
                           padding = 1,
                           bias = False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes,
                           kernel_size = 3,
                           stride = 1,
                           padding = 1,
                           bias = False)
    self.bn2 = nn.BatchNorm2d(planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != planes:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, planes,
                           kernel_size = 3,
                           stride = stride,
                           padding = 1,
                           bias = False),
          nn.BatchNorm2d(planes))
      
  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out

class Net(nn.Module):
  
    def __init__(self): 
        super(Net, self).__init__() 

        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16,
                                kernel_size = 3,
                                stride = 1,
                                padding = 1,
                                bias = False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(BasicBlock, 16, 2, stride = 1)
        self.layer2 = self._make_layer(BasicBlock, 32, 2, stride = 2)
        self.layer3 = self._make_layer(BasicBlock, 64, 2, stride = 2)
        self.layer4 = self._make_layer(BasicBlock, 128, 2, stride = 2)
        self.linear = nn.Linear(128, 21)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
          layers.append(block(self.in_planes, planes, stride))
          self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = F.avg_pool2d(x, 8)
        x = x.view(x.size(0), -1)
        x = self.linear(x)

        return x

model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

* 모델 학습을 위한 함수
* 모델 학습, 평가를 위한 가이드라인 코드입니다. 꼭 이 코드를 사용하지는 않으셔도 됩니다 !!

In [8]:
def train(model, train_loader, optimizer):
    model.train()  
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss =  criterion(output, target) # Cross Entropy Loss 사용했습니다
        loss.backward()
        optimizer.step()

* 모델 평가를 위한 함수

In [9]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0 
    correct = 0   
    
    with torch.no_grad(): 
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)  
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item() 
   
    test_loss /= len(test_loader.dataset) 
    test_accuracy = 100. * correct / len(test_loader.dataset) 
    return test_loss, test_accuracy  

* 모델 학습을 실행하기

In [10]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0  
    best_model_wts = copy.deepcopy(model.state_dict()) 
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()  
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)
        
        if best_acc <= val_acc: 
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since 
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))   
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 

    model.load_state_dict(best_model_wts)  
    return model
 

base = train_baseline(model_base ,train_loader, val_loader, optimizer)  	
torch.save(base,'baseline.pt')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


-------------- epoch 1 ----------------
train Loss: 0.0215, Accuracy: 9.63%
val Loss: 0.0215, Accuracy: 9.87%
Completed in 0m 50s
-------------- epoch 2 ----------------
train Loss: 0.0021, Accuracy: 83.44%
val Loss: 0.0022, Accuracy: 83.18%
Completed in 0m 50s
-------------- epoch 3 ----------------
train Loss: 0.0035, Accuracy: 72.42%
val Loss: 0.0036, Accuracy: 71.04%
Completed in 0m 49s
-------------- epoch 4 ----------------
train Loss: 0.0015, Accuracy: 86.97%
val Loss: 0.0017, Accuracy: 85.09%
Completed in 0m 49s
-------------- epoch 5 ----------------
train Loss: 0.0011, Accuracy: 90.79%
val Loss: 0.0013, Accuracy: 88.51%
Completed in 0m 50s
-------------- epoch 6 ----------------
train Loss: 0.0016, Accuracy: 86.83%
val Loss: 0.0018, Accuracy: 85.61%
Completed in 0m 49s
-------------- epoch 7 ----------------
train Loss: 0.0008, Accuracy: 92.96%
val Loss: 0.0011, Accuracy: 90.44%
Completed in 0m 49s
-------------- epoch 8 ----------------
train Loss: 0.0009, Accuracy: 92.03%
v

## 2. Transfer Learning 모델 학습

* Transfer Learning을 위한 준비
* Transfer Learning이 익숙하지 않으신 분들은 PyTorch에서 제공하는 https://9bow.github.io/PyTorch-tutorials-kr-0.3.1/beginner/transfer_learning_tutorial.html 을 참고하세요 :)

In [11]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    
    'val': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
}

In [12]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

train_dataset_resnet = ImageFolder(root='./splitted/train', transform = data_transforms['train'])
val_dataset_resnet = ImageFolder(root='./splitted/val', transform = data_transforms['val'])

In [13]:
from torch.utils.data import DataLoader

train_loader_resnet = torch.utils.data.DataLoader(train_dataset_resnet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader_resnet = torch.utils.data.DataLoader(val_dataset_resnet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### Pre-Trained Model 불러오기
- 저는 ResNet50을 사용했는데, 코랩 기준으로 다른 ResNet계열이나 DenseNet 정도까지는 큰 무리 없이 훈련할 수 있습니다. Unfreeze layer 수가 적으면 다른 모델도 사용할 수 있을 것입니다.
- 한 가지 모델을 선택해서 Transfer Learning을 해 주세요 !!

In [14]:
from torchvision import models

resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 21)
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)

from torch.optim import lr_scheduler  #scheduler는 사용하지 않으셔도 됩니다 (선택사항)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)  #7에폭마다 learning rate를 조절하는 역할을 합니다

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


* Pre-Trained Model의 일부 Layer Freeze하기 (resnet 기준입니다 !!)

In [15]:
ct = 0
for child in resnet.children():
    ct+= 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

* Fine Tuning을 진행해주세요 ~!

In [16]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss =  criterion(output, target) # Cross Entropy Loss 사용했습니다
        loss.backward()
        optimizer.step()

In [17]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
 
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
   
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [18]:
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)
        
        if best_acc <= val_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    model.load_state_dict(best_model_wts)
    return model

resnet50 = train_baseline(resnet, train_loader_resnet, val_loader_resnet, optimizer_ft)
torch.save(resnet50,'resnet50.pt')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


-------------- epoch 1 ----------------
train Loss: 0.2603, Accuracy: 93.10%
val Loss: 0.2946, Accuracy: 92.30%
Completed in 0m 59s
-------------- epoch 2 ----------------
train Loss: 0.2105, Accuracy: 94.46%
val Loss: 0.2727, Accuracy: 92.82%
Completed in 0m 58s
-------------- epoch 3 ----------------
train Loss: 0.1404, Accuracy: 95.61%
val Loss: 0.1998, Accuracy: 94.57%
Completed in 0m 58s
-------------- epoch 4 ----------------
train Loss: 0.0547, Accuracy: 98.14%
val Loss: 0.0971, Accuracy: 97.07%
Completed in 0m 59s
-------------- epoch 5 ----------------
train Loss: 0.0468, Accuracy: 98.57%
val Loss: 0.0884, Accuracy: 97.41%
Completed in 0m 58s
-------------- epoch 6 ----------------
train Loss: 0.0355, Accuracy: 98.98%
val Loss: 0.0791, Accuracy: 97.44%
Completed in 0m 58s
-------------- epoch 7 ----------------
train Loss: 0.0928, Accuracy: 97.63%
val Loss: 0.1806, Accuracy: 95.93%
Completed in 0m 58s
-------------- epoch 8 ----------------
train Loss: 0.0199, Accuracy: 99.27%

## 모델 평가

* 모델 평가를 위해서는 평가 데이터 또한 전처리를 해주어야 합니다.
* validation 데이터와 동일하게 전처리를 해 주세요 ~!!

* 베이스라인 모델 평가를 위한 전처리하기

In [19]:
transform_base = transforms.Compose([transforms.Resize([64,64]), transforms.ToTensor()])
test_base = ImageFolder(root='./splitted/test',transform = transform_base)
test_loader_base = torch.utils.data.DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


* Transfer Learning모델 평가를 위한 전처리하기

In [20]:
transform_resNet = transforms.Compose([
        transforms.Resize([64,64]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

test_resNet = ImageFolder(root='./splitted/test', transform=transform_resNet)
test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### 성능 평가하기
* 저는 여기서 accuracy만을 평가했지만, 분류 모델이기에 다양한 방법의 평가가 가능합니다.
* Confusion Matrix를 이용한 비교도 가능하고, 한 작물에 해당하는 클래스가 여러개인 다중 분류에서 F1-score를 계산하는것도 의미가 있을 것입니다. 
* 다양한 시도를 하시는 분께 가산점 드리겠습니다 :):)

* 베이스라인 모델 성능 평가하기

In [21]:
baseline=torch.load('baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('test acc:  ', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test acc:   98.563593627579


* Transfer Learning 모델 성능 평가하기

In [22]:
resnet50=torch.load('resnet50.pt')
resnet50.eval()
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('test acc:  ', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test acc:   98.79864194306607


* 두 모델의 성능을 비교 평가하는 설명을 작성해주세요 ~!!

확실히 전이학습을 할 경우에 모델이 조금 더 빨리 전역최적점에 도달한다는 것을 알 수 있다. 하지만 학습 데이터가 매우 상이한 특성을 가진다면 이는 달라질 수 있을 것이다. 그러나 현재 데이터를 기준으로는 전이 학습시 빠르게 전역최적점에 도달했다. 또한 베이스라인 모델의 경우 renet16으로 설정했고 전이학습시 사용했던 모델은 resnet50인데 이번 데이터의 경우에는 층을 더 깊게 쌓을 수록 성능이 더 높아진다는 것을 베이스라인과 전이학습시 활용한 모델의 성능의 비교를 통해 알 수 있었다.

실제 테스트 데이터의 경우에는 학습 데이터로 처음부터 끝까지 학습시킨 베이스라인 모델보다 전이학습을 활용한 모델이 성능이 더 높았다. 전이학습 모델의 경우 일부 레이어의 가중치를 고정시킨 후 학습하였는데 이는 실제 ResNet50을 학습시킨 모델의 데이터와 현재 주어진 Task의 경우 데이터의 특성이 어느정도 비슷하기 때문인 것으로 생각되며 또한 Data Augmentation을 진행하여 조금 더 Nois에 잘 대응하게 만들었기 때문인 것으로 생각된다.

결론은 전이학습을 활용한 모델이 성능이 높았는데 이는 기존 ResNet50을 학습시킨 모델의 데이터와 현재 주어진 Task의 경우 데이터의 특성이 어느정도 비슷하며, Data Augmentation과 정규화를 통해서 Nois에 조금 더 잘 대응하도록 하였으며, 층을 조금 더 깊이 쌓아서 이미지의 특성을 조금 더 잘 찾을 수 있게 하여 성능이 조금 더 높아진 것으로 생각된다.